In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
train_features = pd.read_csv("training_set_features.csv")
train_labels = pd.read_csv("training_set_labels.csv")
test_features = pd.read_csv("test_set_features.csv")
submission_format = pd.read_csv("submission_format.csv")
train_data = pd.merge(train_features, train_labels, on='respondent_id')
train_data.fillna(method='ffill', inplace=True)
train_data = pd.get_dummies(train_data, columns=['age_group', 'education', 'race', 'sex', 'income_poverty',
                                                 'marital_status', 'rent_or_own', 'employment_status',
                                                 'hhs_geo_region', 'census_msa', 'employment_industry',
                                                 'employment_occupation'])
X = train_data.drop(['respondent_id', 'xyz_vaccine', 'seasonal_vaccine'], axis=1)
y = train_data[['xyz_vaccine', 'seasonal_vaccine']]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred_proba = model.predict_proba(X_val)
roc_auc = roc_auc_score(y_val, y_pred_proba, average='macro')
print("ROC AUC Score:", roc_auc)
test_features.fillna(method='ffill', inplace=True)
test_features = pd.get_dummies(test_features, columns=['age_group', 'education', 'race', 'sex', 'income_poverty',
                                                       'marital_status', 'rent_or_own', 'employment_status',
                                                       'hhs_geo_region', 'census_msa', 'employment_industry',
                                                       'employment_occupation'])
test_predictions = model.predict_proba(test_features.drop('respondent_id', axis=1))
submission = pd.DataFrame({
    'respondent_id': test_features['respondent_id'],
    'xyz_vaccine': test_predictions[:, 0],
    'seasonal_vaccine': test_predictions[:, 1]
})
submission.to_csv('submission.csv', index=False)


ParserError: Error tokenizing data. C error: EOF inside string starting at row 18090